# Nh5 Event Model

## Notes on building an event model based in hdf5, pytabes and pandas

In [1]:
from __future__ import print_function

### GATE event model

GATE event model (EM) can be exposed to python (via pyROOT). On the other hand:

1. Since the GATE objects were designed in C++, the objects exposed to python are "C++ translations" performed by pyROOT. This comes at the cost of loosing reflection/introspection. pyROOT objects can only be manipulated using their C++ interfaces, which are often not obviously translated to python (maps is a good example). 

2. GATE objects are stored using ROOT. Depending on your faith you may consider this a blessing or a curse. 

3. A description of GATE EVM is here:

http://next.ific.uv.es:8888/nextsw/dstbuilder/wikis/EventModel

## Nh5 Event Model

### Raw Data objects

#### True Waveforms (TWF), Raw Waveforms (RWF),  Baseline restored Waveforms (BRWF) 

1. TWF are waveforms of photoelectrons (pes) produced by NEXUS and include detector response simulated by Geant-4, propagation of drift electrons, generation and propagation of light, geometrical effects in the sensors (windows) and the QE/PDE of the sensors (PMTs and SiPMs). TWF do not include the response of the electronics. 

2. For detailed studies of the Energy Plane FEE, TWF1ns (bins of 1 ns) in photoelectrons (pes) are neeed. We refer to these as Monte Carlo Raw Data (MCRD). The MCRD is the input to the Simulation of the Energy Plane Response (SIERPE), which convolutes the TWF1ns with the response of the Energy plane FEE and outputs TWF25ns (in adc counts), corresponding to those that are produced by the DAQ if no Baseline Restoration is performed in the FPGA. Those waveforms are called Raw Waveforms (RWF). RWF show the negative swing introduced by the FEE and need to be corrected by digital base line restoration (DBLR). 

3. DBLR can be implemented offline or online, in the DAQ FPGAs. DBLR takes TEF25ns as input and produces corected waveforms (CWF), where the negative swing has been corrected. 

http://localhost:8888/notebooks/Notebooks/SIERPE.ipynb

#### Raw Data flow in the MC

1. For detailed MC simulation of the EP one needs to produce (using NEXUS) MCRD files. In spite of the fact that TWF1ns are very large (thus MCRD files weight about 10 MB per event if not compressed), compression is very effective here, since most of the WF contents are zeros. The actual size of the event is of the order of 250 kb. 
2. MCRD are passed by the SIERPE simulation which produces Raw Waveforms (RWF) files. The resulting TWF25ns show the FEE-induced negative swing. In spite of the fact that TWF25ns have 25 times less elements than TWF1ns, the resulting files, at the same level of compression are heavier, because compression is less effective (there are many numbers close to zero but not exactly zero, due to noise). The file weight about 0.7 kb per event. 
3. RWF are passed by DBLR which produces corrected (baseline restored) Waveforms. The resulting CWF are also stored in the RWF file. Notice that NEXUS can produce directly CWF, corresponding to the case where DBLR is performed directly in the FPGA (in this case the DAQ also produces CWF). 

### Reconstructed Objects

1. Reconstructed objects (RO) are produced by sequencial processing of the data. 
2. In the EP, the relevant actions, starting from the BRWF are the calculation of the energy in each PMT (in pes) and the construction of time-series (after zero suppression) describing S1 and S2 signals
3. In the TP, the relevant actions is the construction of SiPM hits asociated to the S2 signals. 
4. RO need to be expressed in the Nh5 DST files. 